In [1]:
import pandas as pd

# pip install -U pandas pyarrow
df = pd.read_parquet("/projectnb/rlhf/mingyuc/DisCO/datasets/maze/test.parquet")   # 若是 S3 路径同样支持


In [2]:
chat_list = df['extra_info'].apply(lambda x: x['chat']).tolist( )  # 提取 'chat' 字段

In [4]:
from vllm import LLM, SamplingParams

#del llm

# 替换为你的本地模型路径
local_model_path = "/projectnb/rlhf/mingyuc/DisCO/exp/qwen_1.5b/global_step_20"

#local_model_path = "/projectnb/rlhf/mingyuc/DisCO/exp/qwen1.5/global_step_78"

#local_model_path = "Qwen/Qwen2.5-1.5B"

llm = LLM(model=local_model_path)


INFO 07-23 00:04:36 config.py:1670] Downcasting torch.float32 to torch.float16.


WARNING 07-23 00:04:40 arg_utils.py:953] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 07-23 00:04:40 config.py:1005] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 07-23 00:04:40 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='/projectnb/rlhf/mingyuc/DisCO/exp/qwen_1.5b/global_step_20', speculative_config=None, tokenizer='/projectnb/rlhf/mingyuc/DisCO/exp/qwen_1.5b/global_step_20', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.28s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:05<00:00,  3.29s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:05<00:00,  2.99s/it]



INFO 07-23 00:04:47 model_runner.py:1071] Loading model weights took 2.9104 GB
INFO 07-23 00:04:47 gpu_executor.py:122] # GPU blocks: 82897, # CPU blocks: 9362
INFO 07-23 00:04:47 gpu_executor.py:126] Maximum concurrency for 131072 tokens per request: 10.12x
INFO 07-23 00:04:50 model_runner.py:1402] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-23 00:04:50 model_runner.py:1406] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-23 00:04:59 model_runner.py:1530] Graph capturing finished in 10 secs.


In [6]:
sampling_params = SamplingParams(temperature=0.6, max_tokens=6)
tokenizer = llm.get_tokenizer()

index = 1


chat_exp = chat_list[3]
print(len(chat_exp))
test_chat = chat_exp[:index]
chat_str = tokenizer.apply_chat_template(test_chat, add_generation_prompt=True, tokenize=False)
chat_str = chat_str.replace(
    "<|im_start|>system\nYou are a helpful assistant.<|im_end|>",
    "<|im_start|>system\nYou are an intelligent agent navigating a maze.\nAt each step, you receive an observation of four adjacent cells, described by their coordinates and whether they are 'path' or 'wall'.\nYou must choose exactly one adjacent cell that is a valid 'path' or 'exit' and move into it.\nAlways move efficiently toward the goal.\nOutput your next move as a single coordinate in the format (row, col).\nDo not explain or repeat the input — just return the next move.\n<|im_end|>"
)
output = llm.generate(chat_str, sampling_params)
print(output[0].outputs[0].text)
print(chat_exp[index-1:index+1])

result = chat_exp[index]['content'] in output[0].outputs[0].text
print(result)



76


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 21.87it/s, est. speed input: 3090.14 toks/s, output: 131.45 toks/s]

(7, 3):
[{'content': '(7, 2): path, (7, 4): path, (6, 3): path, (8, 3): wall', 'role': 'user'}
 {'content': '(6, 3)', 'role': 'assistant'}]
False


In [ ]:
sampling_params = SamplingParams(temperature=0.0, max_tokens=6)
tokenizer = llm.get_tokenizer()
chat_strs = []
targets = []

for chat_exp in chat_list[:100]:
    for index in range(1, len(chat_exp), 2):
        test_chat = chat_exp[:index]
        chat_str = tokenizer.apply_chat_template(test_chat, add_generation_prompt=True, tokenize=False)
        chat_str = chat_str.replace(
            "<|im_start|>system\nYou are a helpful assistant.<|im_end|>",
            "<|im_start|>system\nYou are an intelligent agent navigating a maze.\nAt each step, you receive an observation of four adjacent cells, described by their coordinates and whether they are 'path' or 'wall'.\nYou must choose exactly one adjacent cell that is a valid 'path' or 'exit' and move into it.\nAlways move efficiently toward the goal.\nOutput your next move as a single coordinate in the format (row, col).\nDo not explain or repeat the input — just return the next move.\n<|im_end|>"
        )
        chat_strs.append(chat_str)
        targets.append(chat_exp[index]['content'])

# 并行推理
outputs = llm.generate(chat_strs, sampling_params)

# 统计正确率
total = len(outputs)
correct = 0
for i, output in enumerate(outputs):
    result = targets[i] in output.outputs[0].text
    if result:
        correct += 1

accuracy = correct / total if total > 0 else 0
print(f"总数: {total}, 正确数: {correct}, 正确率: {accuracy:.4f}")

Processed prompts: 100%|██████████| 3412/3412 [01:29<00:00, 38.11it/s, est. speed input: 40298.81 toks/s, output: 228.66 toks/s]

总数: 3412, 正确数: 2950, 正确率: 0.8646


: 

In [ ]:
sampling_params = SamplingParams(temperature=0.0, max_tokens=6)
tokenizer = llm.get_tokenizer()
chat_strs = []
targets = []

for chat_exp in chat_list[:100]:
    for index in range(1, len(chat_exp), 2):
        test_chat = chat_exp[:index]
        chat_str = tokenizer.apply_chat_template(test_chat, add_generation_prompt=True, tokenize=False)
        chat_str = chat_str.replace(
            "<|im_start|>system\nYou are a helpful assistant.<|im_end|>",
            "<|im_start|>system\nYou are an intelligent agent navigating a maze.\nAt each step, you receive an observation of four adjacent cells, described by their coordinates and whether they are 'path' or 'wall'.\nYou must choose exactly one adjacent cell that is a valid 'path' or 'exit' and move into it.\nAlways move efficiently toward the goal.\nOutput your next move as a single coordinate in the format (row, col).\nDo not explain or repeat the input — just return the next move.\n<|im_end|>"
        )
        chat_strs.append(chat_str)
        targets.append(chat_exp[index]['content'])

In [ ]:
import pandas as pd

data = []
for chat_exp in chat_list[0:3000]:
    for index in range(1, len(chat_exp), 2):
        test_chat = chat_exp[:index]
        answer = chat_exp[index]['content']
        data.append({
            "extra_info": {
                "question": test_chat,
                "answer": answer
            }
        })

df = pd.DataFrame(data)

In [ ]:
import datasets
from datasets import Dataset
df.to_parquet('/projectnb/rlhf/mingyuc/DisCO/datasets/maze/test_new.parquet',
              engine='pyarrow',           # 推荐，用 Arrow 写更快
              index=False)                # 不把行索引写进去

: 